In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput



In [3]:
role = get_execution_role()
session = sagemaker.Session()


[01/27/25 00:10:02] INFO     Found credentials from IAM Role:                                   ]8;id=390386;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=720468;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[01/27/25 00:10:03] INFO     Found credentials from IAM Role:                                   ]8;id=488196;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=575967;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [4]:

# Training configuration
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=sagemaker.image_uris.retrieve("xgboost", "us-west-2", "1.5-1"),
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path="s3://lpl-client-portfolios/model_output/",
    base_job_name="portfolio-rebalancing-xgboost"
)

# Hyperparameters
xgb_estimator.set_hyperparameters(
    objective="reg:squarederror",
    num_round=100,
    max_depth=5,
    eta=0.2
)


[01/27/25 00:10:28] INFO     Ignoring unnecessary instance type: None.                            ]8;id=26870;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=303654;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=511427;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=705941;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=272107;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=874327;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [5]:

# Start training
xgb_estimator.fit({
    "train": TrainingInput(
        s3_data="s3://lpl-client-portfolios/training_data/synthetic_portfolios.csv",
        content_type="csv"
    )
})



[01/27/25 00:10:33] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=537956;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=419116;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=123567;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=412248;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             portfolio-rebalancing-xgboost-2025-01-27-00-10-33-839                                 

2025-01-27 00:10:35 Starting - Starting the training job...
..25-01-27 00:10:50 Starting - Preparing the instances for training.
..25-01-27 00:11:21 Downloading - Downloading input data.
..25-01-27 00:11:46 Downloading - Downloading the training image.
./miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-01-27 00:12:39.287 ip-10-0-133-235.us-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-01-27 00:12:39.312 ip-10-0-133-235.us-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-01-27:00:12:39:INFO] Imported framework sagemaker_xgboost_container.training
[2025-01-27:00:12:39:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2025-01-27:00:12:3

In [6]:
predictor = xgb_estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="portfolio-rebalancing-endpoint"
)


[01/27/25 00:17:22] INFO     Creating model with name:                                              ]8;id=840554;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=895779;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             portfolio-rebalancing-xgboost-2025-01-27-00-17-22-429                                 

[01/27/25 00:17:23] INFO     Creating endpoint-config with name portfolio-rebalancing-endpoint      ]8;id=511033;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=361129;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name portfolio-rebalancing-endpoint             ]8;id=478721;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=846882;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!